In [5]:
!pip3 install baostock -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 231.8 kB/s eta 0:00:00a 0:00:01


In [2]:
import akshare as ak
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import traceback
import concurrent.futures as cf
from datetime import datetime,timedelta

# 下载股票

In [ ]:
stocks = ak.stock_zh_a_spot()
stocks = stocks.loc[:, ['代码', '名称']]
stocks.to_csv("~/stock/stocks.csv")

In [ ]:
def get_a_daily_stock(stock_code):
    try:
        # 从 1990 年至今获取股票历史数据
        stock_data = ak.stock_zh_a_daily(symbol=stock_code, start_date='19900101', adjust='qfq')
        # 将数据写入 csv 文件
        stock_data.to_csv(f'~/stock/a_daily/{stock_code}.csv', index=False, float_format='%.15f')
    except Exception as e:
        print(traceback.format_exc())
        
def get_a_daily_stocks():

    # 读取包含股票代码的 csv 文件
    stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']
    
    # 创建线程池
    with cf.ThreadPoolExecutor(max_workers=10) as executor:
        
        # 提交任务
        futures = [executor.submit(get_a_daily_stock, stock_code) for stock_code in stock_codes]
        # 等待所有任务完成
        cf.wait(futures)
        
get_a_daily_stocks()

# 下载历史日线数据，按股票代码保存(带换手率等数据)

In [4]:
#period='daily'; choice of {'daily', 'weekly', 'monthly'}
def get_a_daily_stock(stock_code):
    try:
        # 从 1990 年至今获取股票历史数据
        stock_data = ak.stock_zh_a_hist(symbol=stock_code, period='daily',start_date='19900101', adjust='qfq')
        # 将数据写入 csv 文件
        stock_data.to_csv(f'~/stock/a_daily/{stock_code}.csv', index=False, float_format='%.15f')
    except Exception as e:
        print(traceback.format_exc())
        
def get_a_daily_stocks():

    # 读取包含股票代码的 csv 文件
    stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']
    
    # 创建线程池
    with cf.ThreadPoolExecutor(max_workers=50) as executor:
        
        # 提交任务
        futures = [executor.submit(get_a_daily_stock, stock_code[-6:]) for stock_code in stock_codes]
        # 等待所有任务完成
        cf.wait(futures)
        
get_a_daily_stocks()

# 证券类别统计:证券类别｜数量｜成交金额｜总市值｜流通市值

In [ ]:
def stock_szse_summary(arr, date_str):
    df = ak.stock_szse_summary(date=date_str)
    # 指定要选取的字符串列表

    temp_arr = np.full((len(required_categories), 4), None)
    for index,categorie in enumerate(required_categories):
        selected_rows = np.where(df['证券类别'] == categorie)[0]
        selected_df = df.iloc[selected_rows, 1:]
        if selected_df.empty is not True:
            temp_arr[index,:] = selected_df
    arr.append(np.append(np.array([date_str]), temp_arr.ravel()))
    print(date_str)

start_date = datetime.datetime(2008, 1, 3)  # 开始日期
end_date = datetime.datetime.now()  # 结束日期

required_categories = ['股票','主板A股','主板B股','中小板','基金','ETF','LOF','封闭式基金','分级基金','权证','股票权证','债券','国债','公司债','企业债','债券回购','可转换债']
arr = []
futures = []
with cf.ThreadPoolExecutor(max_workers=50) as executor:
    while start_date <= end_date:
        date_str = start_date.strftime('%Y%m%d')  # 将日期格式化为字符串
        futures.append(executor.submit(stock_szse_summary, arr, date_str))
        start_date += datetime.timedelta(days=1)  # 加上一天
    cf.wait(futures)
df = pd.DataFrame(arr)
df.columns = ['日期','股票数量','股票成交金额','股票总市值','股票流通市值','主板A股数量','主板A股成交金额','主板A股总市值','主板A股流通市值','主板B股数量','主板B股成交金额','主板B股总市值','主板B股流通市值','中小板数量','中小板成交金额','中小板总市值','中小板流通市值','基金数量','基金成交金额','基金总市值','基金流通市值','ETF数量','ETF成交金额','ETF总市值','ETF流通市值','LOF数量','LOF成交金额','LOF总市值','LOF流通市值','封闭式基金数量','封闭式基金成交金额','封闭式基金总市值','封闭式基金流通市值','分级基金数量','分级基金成交金额','分级基金总市值','分级基金流通市值','权证数量','权证成交金额','权证总市值','权证流通市值','股票权证数量','股票权证成交金额','股票权证总市值','股票权证流通市值','债券数量','债券成交金额','债券总市值','债券流通市值','国债数量','国债成交金额','国债总市值','国债流通市值','公司债数量','公司债成交金额','公司债总市值','公司债流通市值','企业债数量','企业债成交金额','企业债总市值','企业债流通市值','债券回购数量','债券回购成交金额','债券回购总市值','债券回购流通市值','可转换债券数量','可转换债券成交金额','可转换债券总市值','可转换债券流通市值']
df.to_csv(f'~/stock/szse_summary.csv', index=False, float_format='%.15f')

# 所有指数当天数据（有指数的编码）

In [ ]:
ak.stock_zh_index_spot().to_csv(f'~/stock/stock_zh_index_spot.csv', index=False, float_format='%.15f')

# 指数历史数据

In [ ]:
def stock_zh_index_daily(code):
    stock_zh_index_daily_df = ak.stock_zh_index_daily(symbol=code)
    stock_zh_index_daily_df.to_csv(f'~/stock/index_daily/{code}.csv', index=False, float_format='%.15f')
    
codes = pd.read_csv('~/stock/stock_zh_index_spot.csv')['代码']
futures = []
with cf.ThreadPoolExecutor(max_workers=50) as executor:
    for code in codes:
        futures.append(executor.submit(stock_zh_index_daily, code))
    cf.wait(futures)
    

# 机构调研

In [ ]:
stock_jgdy_tj_em_df = ak.stock_jgdy_tj_em(date="20210128")
stock_jgdy_tj_em_df.to_csv(f'~/stock/jgdy.csv', index=False, float_format='%.15f')
print(stock_jgdy_tj_em_df)

# 机构调研详情（带调研机构和基金经理）

In [ ]:
stock_jgdy_detail_em_df = ak.stock_jgdy_detail_em(date="20080101")
stock_jgdy_detail_em_df.to_csv(f'~/stock/jgdy_detail.csv', index=False, float_format='%.15f')
print(stock_jgdy_detail_em_df)

# 公司主营介绍

In [ ]:
def stock_zygc_em(code, arr):
    stock_zygc_em_df = ak.stock_zygc_em(symbol=code).T.values
    return stock_zygc_em_df.transpose()

# 读取包含股票代码的 csv 文件
stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']

arr = []
# 创建线程池
with cf.ThreadPoolExecutor(max_workers=50) as executor:
    # 提交任务
    futures = [executor.submit(stock_zygc_em, stock_code, arr) for stock_code in stock_codes]
    # 等待所有任务完成
    res_list = [f.result() for f in cf.as_completed(futures)]
    arr = np.vstack(res_list)
df = pd.DataFrame(arr)
df.columns = ['股票代码','报告日期','分类类型','主营构成','主营收入','收入比例','主营成本','成本比例','主营利润','利润比例','毛利率']
df.to_csv(f'~/stock/zygc_dfcf.csv', index=False, float_format='%.15f')


# 上市公司股权质押

In [ ]:
df = ak.stock_gpzy_pledge_ratio_detail_em()
df.to_csv(f'~/stock/gpzy_pledge_ratio_detail.csv', index=False, float_format='%.15f')

# 北向净流入["沪股通", "深股通", "北上"]

In [ ]:
df = ak.stock_hsgt_north_net_flow_in_em(symbol="沪股通")
df.to_csv(f'~/stock/stock_hsgt_north_net_flow_in_lgt.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_north_net_flow_in_em(symbol="深股通")
df.to_csv(f'~/stock/stock_hsgt_north_net_flow_in_sgt.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_north_net_flow_in_em(symbol="北上")
df.to_csv(f'~/stock/stock_hsgt_north_net_flow_in_bs.csv', index=False, float_format='%.15f')

# 北向累计净流入["沪股通", "深股通", "北上"]

In [ ]:
df = ak.stock_hsgt_north_acc_flow_in_em(symbol="沪股通")
df.to_csv(f'~/stock/stock_hsgt_north_acc_flow_in_hgt.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_north_acc_flow_in_em(symbol="深股通")
df.to_csv(f'~/stock/stock_hsgt_north_acc_flow_in_sgt.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_north_acc_flow_in_em(symbol="北上")
df.to_csv(f'~/stock/stock_hsgt_north_acc_flow_in_bs.csv', index=False, float_format='%.15f')

# 每日净流入个股统计 【"北向持股", "沪股通持股", "深股通持股", "南向持股"】

In [ ]:
df = ak.stock_hsgt_stock_statistics_em(symbol="北向持股", start_date="20000101", end_date="20230527")
df.to_csv(f'~/stock/stock_hsgt_stock_statistics_bxzj.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_stock_statistics_em(symbol="沪股通持股", start_date="20000101", end_date="20230527")
df.to_csv(f'~/stock/stock_hsgt_stock_statistics_hgtcgj.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_stock_statistics_em(symbol="深股通持股", start_date="20000101", end_date="20230527")
df.to_csv(f'~/stock/stock_hsgt_stock_statistics_sgtcgj.csv', index=False, float_format='%.15f')


# 沪深港通历史数据 【 "沪股通", "深股通", "港股通沪", "港股通深"】

In [ ]:
df = ak.stock_hsgt_hist_em(symbol="沪股通")
df.to_csv(f'~/stock/stock_hsgt_hist_hgt.csv', index=False, float_format='%.15f')

df = ak.stock_hsgt_hist_em(symbol="深股通")
df.to_csv(f'~/stock/stock_hsgt_hist_sgt.csv', index=False, float_format='%.15f')

# 沪深港通持股-个股

In [ ]:
def get_a_daily_stock(stock_code):
    df = ak.stock_hsgt_individual_em(stock=stock_code[2:])
    # 将数据写入 csv 文件
    df.to_csv(f'~/stock/a_sgt_daily/{stock_code}.csv', index=False, float_format='%.15f')
        
def get_a_daily_stocks():

    # 读取包含股票代码的 csv 文件
    stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']
    
    # 创建线程池
    with cf.ThreadPoolExecutor(max_workers=10) as executor:
        
        # 提交任务
        futures = [executor.submit(get_a_daily_stock, stock_code) for stock_code in stock_codes]
        # 等待所有任务完成
        cf.wait(futures)
        
get_a_daily_stocks()

# 沪深港通持股-个股详情

In [ ]:
def get_daily(stock_code):
    df = ak.stock_hsgt_individual_detail_em(symbol=stock_code[2:], start_date="20100830", end_date="20230528")
    # 将数据写入 csv 文件
    df.to_csv(f'~/stock/a_hsgt_daily/{stock_code}.csv', index=False, float_format='%.15f')
        
def get_all():

    # 读取包含股票代码的 csv 文件
    stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']
    
    # 创建线程池
    with cf.ThreadPoolExecutor(max_workers=10) as executor:
        
        # 提交任务
        futures = [executor.submit(get_daily, stock_code) for stock_code in stock_codes]
        # 等待所有任务完成
        cf.wait(futures)
        
get_all()

# 业绩报表

In [ ]:
def get_daily(date_file_name):
    df = ak.stock_yjbb_em(date_file_name)
    # 将数据写入 csv 文件
    df.to_csv(f'~/stock/a_yjbb/{date_file_name}.csv', index=False, float_format='%.15f')

date_str = ["0331", "0630", "0930", "1231"] 
for year in range(2020, 2023):
    for md in date_str:
        get_daily(f'{year}{md}')

# 业绩快报

In [ ]:
def get_daily(date_file_name):
    df = ak.stock_yjkb_em(date_file_name)
    # 将数据写入 csv 文件
    df.to_csv(f'~/stock/a_yjkb/{date_file_name}.csv', index=False, float_format='%.15f')

date_str = ["0331", "0630", "0930", "1231"] 
for year in range(2020, 2023):
    for md in date_str:
        get_daily(f'{year}{md}')

# 业绩预告

In [ ]:
def get_daily(date_file_name):
    df = ak.stock_yjyg_em(date_file_name)
    # 将数据写入 csv 文件
    df.to_csv(f'~/stock/a_yjyg/{date_file_name}.csv', index=False, float_format='%.15f')

date_str = ["0331", "0630", "0930", "1231"] 
for year in range(2016, 2023):
    for md in date_str:
        get_daily(f'{year}{md}')

# 龙虎榜

In [ ]:
df = ak.stock_lhb_detail_em(start_date="20000101", end_date="20230417")
df.to_csv(f'~/stock/a_lhb.csv', index=False, float_format='%.15f')

# 股票热度-问财

In [ ]:
def get_detail(date_str):
    df = ak.stock_zt_pool_em(date=date_str)
    df.to_csv(f'~/stock/a_zt_pool/{date_str}.csv', index=False, float_format='%.15f')

start_date = datetime(2000, 1, 1)  # 开始日期
end_date = datetime.now()  # 结束日期
while start_date <= end_date:
    get_detail(start_date.strftime('%Y%m%d'))
    start_date += timedelta(days=1)  # 加上一天


# 历史分笔（最近两年）

In [ ]:
def get_daily(stock_code):
    try:
        # 从 1990 年至今获取股票历史数据
        stock_data = ak.stock_zh_a_daily(symbol=stock_code, start_date='19900101', adjust='hfq')
        # 将数据写入 csv 文件
        stock_data.to_csv(f'~/stock/a_daily/{stock_code}.csv', index=False, float_format='%.15f')
    except Exception as e:
        print(traceback.format_exc())
        
def get_all():

    # 读取包含股票代码的 csv 文件
    stock_codes = pd.read_csv('~/stock/stocks.csv')['代码']
    
    # 创建线程池
    with cf.ThreadPoolExecutor(max_workers=10) as executor:
        
        # 提交任务
        futures = [executor.submit(get_daily, stock_code) for stock_code in stock_codes]
        # 等待所有任务完成
        cf.wait(futures)
        
get_all()